# Example 3: Extended usage and output diagnostics

In this notebook, we demonstrate extended usage of PyVBMC. We will take a brief look at PyVBMC's diagnostic output, and show you how to save the results of optimization to disk.

This notebook is Part 3 of a series of notebooks in which we present various example usages for VBMC with the PyVBMC package.

In [2]:
import numpy as np
import scipy.stats as scs
from scipy.optimize import minimize
from pyvbmc.vbmc import VBMC
from pyvbmc.formatting import format_dict
import dill

## 1. Model definition and setup

For demonstration purposes, we will run PyVBMC with a restricted budget of function evaluations, insufficient to achieve convergence. Then we will inspect the output diagnostics, and resume optimization.

We use a higher-dimensional analogue of the same toy target function in Example 1, a broad [Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function) in $D = 4$.

In [3]:
D = 4  # A four-dimensional problem
prior_mu = np.zeros(D)
prior_var = 3 * np.ones(D)


def log_prior(theta):
    """Multivariate normal prior on theta."""
    cov = np.diag(prior_var)
    return scs.multivariate_normal(prior_mu, cov).logpdf(theta)

The likelihood function of your model will in general depend on the observed data. This data can be fixed as a global variable, as we did directly above for `prior_mu` and `prior_var`. It can also be defined by a default second argument: to `pyvbmc` there is no difference so long as the function can be called with only a single argument (the parameters `theta`):

In [4]:
def log_likelihood(theta, data=np.ones(D)):
    """D-dimensional Rosenbrock's banana function."""
    # In this simple demo the data just translates the parameters:
    theta = np.atleast_2d(theta)
    theta = theta + data

    x, y = theta[:, :-1], theta[:, 1:]
    return -np.sum((x**2 - y) ** 2 + (x - 1) ** 2 / 100, axis=1)


def log_joint(theta, data=np.ones(D)):
    """log-density of the joint distribution."""
    return log_likelihood(theta, data) + log_prior(theta)

In [5]:
LB = np.full(D, -np.inf)  # Lower bounds
UB = np.full(D, np.inf)  # Upper bounds
PLB = np.full(D, prior_mu - np.sqrt(prior_var))  # Plausible lower bounds
PUB = np.full(D, prior_mu + np.sqrt(prior_var))  # Plausible upper bounds

In a typical inference scenario, we recommend starting from a "good" point (i.e. one near the mode). We can run a  quick preliminary optimization, though a more extensive optimization would not harm.

In [6]:
np.random.seed(41)
x0 = np.random.uniform(PLB, PUB)  # Random point inside plausible box
x0 = minimize(
    lambda t: -log_joint(t),
    x0,
    bounds=[
        (-np.inf, np.inf),
        (-np.inf, np.inf),
        (-np.inf, np.inf),
        (-np.inf, np.inf),
    ],
).x
np.random.seed(42)

In [7]:
# Limit number of function evaluations
options = {
    "max_fun_evals": 10 * D,
}
# We can specify either the log-joint, or the log-likelihood and log-prior.
# In other words, the following lines are equivalent:
vbmc = VBMC(
    log_likelihood,
    x0,
    LB,
    UB,
    PLB,
    PUB,
    options=options,
    log_prior=log_prior,
)
# vbmc = VBMC(
#     log_joint,
#     x0, LB, UB, PLB, PUB, user_options=options,
# )

Reshaping x0 to row vector.
Reshaping lower bounds to (1, 4).
Reshaping upper bounds to (1, 4).
Reshaping plausible lower bounds to (1, 4).
Reshaping plausible upper bounds to (1, 4).


(PyVBMC expects the bounds to be `(1, D)` row vectors, and the initial point(s) to be of shape `(n, D)`, but it will accept and re-shape vectors of shape `(D,)` as well.)

## 2. Running the model and checking convergence diagnostics

Now we run PyVBMC with a very small budget of 40 function evaluations:

In [8]:
vp, elbo, elbo_sd, success_flag, info = vbmc.optimize()

Beginning variational optimization assuming EXACT observations of the log-joint.
 Iteration  f-count    Mean[ELBO]    Std[ELBO]    sKL-iter[q]   K[q]  Convergence  Action
     0         10          -3.72         2.61    319790.86        2        inf     start warm-up
     1         15          -3.96         1.53        12.50        2        inf     
     2         20          -4.11         2.33         0.37        2       14.5     
     3         25          -4.83         1.20        12.01        2        206     
     4         30          -4.29         0.89         4.10        2         73     
     5         35          -5.91         1.54         8.47        2        152     
     6         40          -2.24         2.92         4.45        2       96.1     
   inf         40          -4.81         2.33         1.07       50        152     finalize
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -4.807 +/-2.330.
Caution: Re

PyVBMC is warning us that convergence is doubtful. We can look at the output for more information and diagnostics.

In [9]:
print(success_flag)

False


`False` means that PyVBMC has not converged to a stable solution within the given number of function evaluations.

In [10]:
print(format_dict(info))

{
    'function': '<function VBMC.__init__.<locals>.log_joint at 0x1507b3280>',
    'problem_type': 'unconstrained',
    'iterations': 6,
    'func_count': 40,
    'best_iter': 5,
    'train_set_size': 35,
    'components': 50,
    'r_index': 151.75094234172184,
    'convergence_status': 'no',
    'overhead': nan,
    'rng_state': 'rng',
    'algorithm': 'Variational Bayesian Monte Carlo',
    'version': '0.1.0',
    'message': 'Inference terminated: reached maximum number of function evaluations options.max_fun_evals.',
    'elbo': -4.8065010675165105,
    'elbo_sd': 2.329618254554614,
}


In the `info` dictionary:
- the `convergence_status` field says 'no' (probable lack of convergence);
- the reliability index `r_index` is 3.68, (should be less than 1).
Our diagnostics tell us that this run has not converged, suggesting to increase the budget.

Note that convergence to a solution does not mean that it is a _good_ solution. You should always check the returned variational posteriors, and ideally should compare across multiple runs of PyVBMC.

## 3. Saving results

We can also save the `VBMC` instance to disk and reload it later, in order to check the results and convergence diagnostics, sample from the posterior, or resume the optimization from checkpoint etc. If you are only interested in the final (best) variational solution, as opposed to the full iteration history of the optimization, then you may wish to pickle only the final `VariationalPosterior` instead.

In [11]:
with open("vbmc_test_save.pkl", "wb") as f:
    dill.dump(vbmc, f)

with open("vbmc_test_save.pkl", "rb") as f:
    vbmc = dill.load(f)

## 4. Resume the optimization process

Now we could resume the optimization process of PyVBMC by increasing maximum number of function evaluations. This default is $50(d+2)$, where $d$ is the dimension of the parameter space (though this example will not use the full budget):

In [12]:
iteration = vbmc.iteration  # continue from specified iteration, here it's the last iteration
assert 1 <= iteration <= vbmc.iteration

## Set states for VBMC right before the specified iteration
vbmc.gp = vbmc.iteration_history["gp"][iteration - 1]
vbmc.vp = vbmc.iteration_history["vp"][iteration - 1]
vbmc.function_logger = vbmc.iteration_history[
    "function_logger"
][iteration - 1]
vbmc.optim_state = vbmc.iteration_history["optim_state"][
    iteration - 1
]
vbmc.hyp_dict = vbmc.optim_state["hyp_dict"]
vbmc.iteration = iteration - 1

for k, v in vbmc.iteration_history.items():
    try:
        vbmc.iteration_history[k] = vbmc.iteration_history[k][
            :iteration
        ]
    except TypeError:
        pass
# (Optionally) Set random state for reproducibility, note that it can only 
# reproduce exactly the same optimization process when VBMC's options are not updated
random_state = vbmc.iteration_history["random_state"][
                    iteration - 1
                ]
np.random.set_state(random_state)

We need to update the options. Here we increase `max_fun_evals` and resume the optimization process.

In [13]:
options = {
    "max_fun_evals": 50 * (D+2),
}
vbmc.options.is_initialized = False  # Temporarily set to False for updating options
vbmc.options.update(options)
vbmc.options.is_initialized = True

vbmc.is_finished = False
vp, elbo, elbo_sd, success_flag, info = vbmc.optimize()

Beginning variational optimization assuming EXACT observations of the log-joint.
 Iteration  f-count    Mean[ELBO]    Std[ELBO]    sKL-iter[q]   K[q]  Convergence  Action
     6         40          -0.50         4.37         7.81        2        163     
     7         45          -4.84         0.10        11.94        2        214     
     8         50          -4.71         0.04         0.07        2       1.71     
     9         55          -4.66         0.02         0.04        2       0.88     
    10         60          -4.62         0.01         0.02        2      0.501     end warm-up
    11         65          -4.58         0.02         0.05        2       1.02     
    12         70          -4.57         0.01         0.02        2      0.379     
    13         75          -4.50         0.01         0.04        5      0.908     
    14         80          -4.37         0.00         0.07        8       1.54     rotoscale, undo rotoscale
    15         85          -4.28     

In [14]:
print(format_dict(info))

{
    'function': '<function VBMC.__init__.<locals>.log_joint at 0x1507735e0>',
    'problem_type': 'unconstrained',
    'iterations': 23,
    'func_count': 125,
    'best_iter': 23,
    'train_set_size': 122,
    'components': 50,
    'r_index': 0.031330287656699755,
    'convergence_status': 'probable',
    'overhead': nan,
    'rng_state': 'rng',
    'algorithm': 'Variational Bayesian Monte Carlo',
    'version': '0.1.0',
    'message': 'Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.',
    'elbo': -4.117721331713407,
    'elbo_sd': 0.0011513438533648059,
}


With the default budget of function evaluations, we can see that the `convergence_status` is 'probable' and the `r_index` is much less than 1, suggesting convergence has been acheived. We can save the result to file and load it at anytime for verification etc.

In [14]:
with open("vbmc_test_save.pkl", "wb") as f:
    dill.dump(vbmc, f)

with open("vbmc_test_save.pkl", "rb") as f:
    vbmc = dill.load(f)
    
samples, components = vbmc.vp.sample(5)
# `samples` are samples drawn from the variational posterior.
# `components` are the index of the mixture components each
#  sample was drawn from.
print(samples)
print(components)

[[-0.38232762 -0.57451283 -0.01202702  1.00881978]
 [-0.13825953  0.58691027  0.97214509  2.10049461]
 [-0.25420183 -0.55995149 -0.9688158  -1.25871633]
 [-0.68187614  0.04510994 -0.05127354 -0.08940267]
 [-1.33565096 -1.99922658  0.30049789  0.76330246]]
[ 5  6 14  4 24]


## 5. Conclusions

In this notebook, we have given a brief overview of PyVBMC's output diagnostics, and shown how to save and load results and resume optimization from a specific iteration.

In the next notebook, we will illustrate running PyVBMC multiple times in order to validate the results.

## Acknowledgments

Work on the PyVBMC package was funded by the [Finnish Center for Artificial Intelligence FCAI](https://fcai.fi/).